# Log Data Example

<div class="alert alert-block alert-info">
    
This example demonstrates how to import and process data from the [CARWatch App](https://play.google.com/store/apps/details?id=de.fau.cs.mad.carwatch) used to record logs during cortisol awakening response (CAR) assessment studies at home.
  
</div>

## Imports and Helper Functions

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np

from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns

from carwatch.logs import ParticipantLogs, StudyLogs

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
base_path = Path("../example_data")

pprint([f for f in sorted(base_path.glob("*")) if f.is_dir() and not f.name.startswith(".")])

## Log Files from Single Participant (`ParticipantLogs`)

Log files can either be imported by (1) loading the `.zip` files obtained by the *CARWatch App* or (2) by loading a folder from a participant what contains the log files as `.csv` files.

For (1), the file structure is assumed to look like:
```
. (base folder)
├── logs_ABC12.zip
├── logs_DEF34.zip
├── logs_GHI56.zip
├── logs_JKL78.zip
...
```

For (2), the file structure is assumed to look like:
```
. (base folder)
├── ABC12/
│   ├── carwatch_abc12_20200101.csv
│   ├── carwatch_abc12_20200102.csv
│   └── carwatch_abc12_20200103.csv
├── DEF34/
│   ├── carwatch_def34_20200201.csv
│   ├── carwatch_def34_20200202.csv
│   └── carwatch_def34_20200203.csv
...
```

### Import

#### From zip File

This example assumes simply loads a zipped log file from one participant as it was exported by the CARWatch application (case (1) in the explanation above). Thus, we search for files that end with `*.zip`.

In [ ]:
# show contents of the example data folder
log_files = sorted(base_path.joinpath("zip_files").glob("*.zip"))
log_files

Select one file and import the participant logs

**Note:** Per default, the log files are simply loaded from the zip file and not extracted.
If you want to extract the log files from the zip file, you can set `extract_folder=True`. This will create a folder with the same name as the zip file and extract the log files into this folder. If you additionally, want to overwrite existing log files in the folder, you can set `overwrite_unzipped_logs=True`.

In [ ]:
# Load log data from zip file:
zip_path = Path(log_files[0])
log = ParticipantLogs.from_zip_file(zip_path, extract_folder=False, overwrite_unzipped_logs=False)

Print some information about the participant

In [ ]:
print(f"Subject ID: {log.subject_id}")
print(f"Start date: {log.start_date}")
print(f"End date: {log.end_date}")

Print some log metadata

In [ ]:
print(f"App version: {log.app_version}")
print(f"Android Version: {log.android_version}")
print(f"Smartphone Model: {log.phone_model}")
print(f"Smartphone Manufacturer: {log.phone_manufacturer}")

Convert the log files into a dataframe

In [ ]:
df = log.data_as_df()
df.head()

Split Log Data by Sampling Days

In [ ]:
log.split_sampling_days()

#### From Folder

This example assumes that the zipped log files from one participant are already unpacked and stored in one directory (case (2) in the explanation above). Thus, we search for directories that start with `logs_`.

**Note:** While the variant above seems more straightforwards, as you don't have to unzip all log files, this method can be useful when the log files were already unzipped, e.g., because they were manually cleaned.

In [ ]:
log_folders = [f for f in sorted(base_path.joinpath("folders").glob("*")) if f.is_dir() and f.name.startswith("logs")]
log_folders


### Filtering/Selecting

Filter by Action

In [ ]:
log.filter_logs(action="barcode_scanned")

Filter by Date

In [ ]:
log.filter_logs(date="2019-12-06").head()

Select one folder, import the participant logs, and convert them into a dataframe

### Export

In [ ]:
# Load log data from folder
log = ParticipantLogs.from_folder(log_folders[0])
df = log.data_as_df()
df.head()

In [ ]:
log.awakening_times()

In [ ]:
log.sampling_times()

In [ ]:
log.export_times(wide_format=True)

## Log Files from All Participants (`StudyLogs`)

Log files from all participants from a study can be represented by a `StudyLogs` object. This object can be used to load all log files from a study and to process them in a more convenient way.

### Import

#### From Folder

In [ ]:
study_logs = StudyLogs.from_folder(base_path.joinpath("complete/zip_files"))

Study information

In [ ]:
print(f"Participants: {study_logs.participants}")

Iterate through all participants

In [ ]:
for key in study_logs:
    print(study_logs[key])

Convert data to dataframe

#### Export Data

In [ ]:
study_logs.data_as_df()

Export awakening and saliva sampling times

In [ ]:
study_logs.export_times(wide_format=True)

### Study Statistics

In [ ]:
study_logs.get_metadata_stats("android_version")

In [ ]:
fig, axs = study_logs.get_metadata_plot("android_version", figsize=(4,2))
fig.tight_layout()
